### Word Group Prepare For Youtube Tool

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [2]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [3]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [4]:
# language pair
lang_folder = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian

# language detail num
if lang_folder == "English":
    lang_id = "1"
elif lang_folder == "Turkish":
    lang_id = "2"
elif lang_folder == "French":
    lang_id = "3"
elif lang_folder == "Spanish":
    lang_id = "4"
elif lang_folder == "Portuguese":
    lang_id = "5"
elif lang_folder == "Dutch":
    lang_id = "6"
elif lang_folder == "Italian":
    lang_id = "7"
elif lang_folder == "German":
    lang_id = "8"
elif lang_folder == "Arabic":
    lang_id = "9"
else:
    pass

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"Language = {lang_folder}\nLang Id = {lang_id}\nFile Extention = {file_ext}")

Language = Arabic
Lang Id = 9
File Extention = 6


In [5]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Result/2-Word Group Prepare For Youtube Tool"

Path(path).mkdir(parents=True, exist_ok=True)

#### Read Data

In [6]:
parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Data/1-Word Group In Youtube Sentence"

In [7]:
df_28 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.csv")
df_28 = df_28.head(700)
df_28

,search_string,start_time,end_time,sentence,video_id,video_url
0,نعم ماذا عن هذا هنا نعم,381,391,نعم ماذا عن هذا هنا نعم,2l3UajCV1mI,https://www.youtube.com/watch?v=2l3UajCV1mI&t=...
1,هل كان هذا كل ما في,366,368,هل كان هذا كل ما في الأمر,RLydD3uP6NI,https://www.youtube.com/watch?v=RLydD3uP6NI&t=...
2,عن ماذا كان كل ذلك,262,265,عن ماذا كان كل ذلك,rVkXdGORB8U,https://www.youtube.com/watch?v=rVkXdGORB8U&t=...
3,في في هل هذه أنت,4218,4220,مرحبا في في هل هذه أنت,7ONEprXyxcs,https://www.youtube.com/watch?v=7ONEprXyxcs&t=...
4,أنا هنا يا في في,4868,4870,أنا هنا يا في في,7ONEprXyxcs,https://www.youtube.com/watch?v=7ONEprXyxcs&t=...
...,...,...,...,...,...,...
432,و لا,612,614,و لا,Mms3BuNkHBs,https://www.youtube.com/watch?v=Mms3BuNkHBs&t=...
433,و هو,53,58,و هو,KBaejvlMXfY,https://www.youtube.com/watch?v=KBaejvlMXfY&t=53s
434,و هو,172,175,و هو,VR6GurhR33A,https://www.youtube.com/watch?v=VR6GurhR33A&t=...
435,و في,53,56,و في,glrQ18CS24A,https://www.youtube.com/watch?v=glrQ18CS24A&t=53s


In [8]:
df_200 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.csv")
df_200 = df_200.head(700)
df_200

,search_string,start_time,end_time,sentence,video_id,video_url
0,نعم حسنا ليس هناك لا يوجد أحد هنا لا يمكنني,435,440,نعم حسنا ليس هناك لا يوجد أحد هنا لا يمكنني ال...,NCem4xje9Ss,https://www.youtube.com/watch?v=NCem4xje9Ss&t=...
1,ماذا ماذا ماذا أنت ما هي لا لا تفعل أنا آسف,882,887,ماذا ماذا ماذا أنت ما هي لا لا تفعل أنا آسف اا...,HnSK6tiiQZg,https://www.youtube.com/watch?v=HnSK6tiiQZg&t=...
2,أوه اللعنة أوه اللعنة ما الذي يحدث يا إلهي,1036,1039,أوه اللعنة أوه اللعنة ما الذي يحدث يا إلهي,VSx7oL5PC7I,https://www.youtube.com/watch?v=VSx7oL5PC7I&t=...
3,لقد قلت أنت لا يمكنك أن تفعل ذلك بي هناك,1242,1248,لقد قلت أنت لا يمكنك أن تفعل ذلك بي هناك أشياء,gqC0SbiQQc4,https://www.youtube.com/watch?v=gqC0SbiQQc4&t=...
4,لذلك أنا لا أعرف أنني أعتقد إذا كنت تريد,295,299,لذلك أنا لا أعرف أنني أعتقد إذا كنت تريد,gkhlMGmoTWE,https://www.youtube.com/watch?v=gkhlMGmoTWE&t=...
...,...,...,...,...,...,...
558,مع أنه في كل مكان,15,17,مع أنه في كل مكان,fXukytNqkt4,https://www.youtube.com/watch?v=fXukytNqkt4&t=15s
559,يا إلهي و سيدي و,731,739,يا إلهي و سيدي و ربي,IHEwci25a5Q,https://www.youtube.com/watch?v=IHEwci25a5Q&t=...
560,ما إذا كان كل ما,1294,1296,ما إذا كان كل ما تريده,AFqZbxxIGK4,https://www.youtube.com/watch?v=AFqZbxxIGK4&t=...
561,أنا هنا يا في في,4868,4870,أنا هنا يا في في,7ONEprXyxcs,https://www.youtube.com/watch?v=7ONEprXyxcs&t=...


In [9]:
df_1000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.csv")
df_1000 = df_1000.head(700)
df_1000

,search_string,start_time,end_time,sentence,video_id,video_url
0,أشياء يجب ألا تتحدث عنها مع الآخرين على الإطلا...,21,30,هذه سبعة أشياء يجب ألا تتحدث عنها مع الآخرين ع...,FF1qfIe0Mb0,https://www.youtube.com/watch?v=FF1qfIe0Mb0&t=21s
1,أن يعرف ما إذا كنت تحصل على ما يكفي من الطعام ...,2470,2475,أراد أخوك أن يعرف ما إذا كنت تحصل على ما يكفي ...,RLydD3uP6NI,https://www.youtube.com/watch?v=RLydD3uP6NI&t=...
2,حسنا ولكن لن تعمل فقط على نفسك اسمع يا رجل أنت...,1327,1331,حسنا ولكن لن تعمل فقط على نفسك اسمع يا رجل أنت...,Q4PgAY_qXKg,https://www.youtube.com/watch?v=Q4PgAY_qXKg&t=...
3,ولكن اليوم الأول أنت لا تحتاج كل هذا أنا أعرف ...,1873,1877,ولكن اليوم الأول أنت لا تحتاج كل هذا أنا أعرف ...,SJ0tF6gA0cM,https://www.youtube.com/watch?v=SJ0tF6gA0cM&t=...
4,عندما يكون لديك نفس طويل أو عندما تكون تعلم ان...,845,850,عندما يكون لديك نفس طويل أو عندما تكون تعلم ان...,lhfkYiCpTiU,https://www.youtube.com/watch?v=lhfkYiCpTiU&t=...
...,...,...,...,...,...,...
695,تأخذ القليل منا إلى هناك الآن,4094,4120,تأخذ القليل منا إلى هناك الآن,2l3UajCV1mI,https://www.youtube.com/watch?v=2l3UajCV1mI&t=...
696,هذه المرة الأمر صعب بعض الشيء,3641,3644,هذه المرة الأمر صعب بعض الشيء,7fyY6ByIQi0,https://www.youtube.com/watch?v=7fyY6ByIQi0&t=...
697,هذه الأشياء يمكنني القيام بها,2635,2637,هذه الأشياء يمكنني القيام بها,wA0GhxqsAyA,https://www.youtube.com/watch?v=wA0GhxqsAyA&t=...
698,عليهم الوصول إلى الجانب الآخر,2000,2003,عليهم الوصول إلى الجانب الآخر,kFyyHU1u2d4,https://www.youtube.com/watch?v=kFyyHU1u2d4&t=...


In [10]:
df_5000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.csv")
df_5000 = df_5000.head(700)
df_5000

,search_string,start_time,end_time,sentence,video_id,video_url
0,سواء كنت تعتقد أنك تستطيع ذلك أو إذا كنت تعتقد...,39,54,قال هنري فورد سواء كنت تعتقد أنك تستطيع ذلك أو...,tEuQpz6cC0U,https://www.youtube.com/watch?v=tEuQpz6cC0U&t=39s
1,التقدم في العمر ليس بالأمر الجيد دائما أغلب ال...,277,288,التقدم في العمر ليس بالأمر الجيد دائما أغلب ال...,NbA_c5PoYMo,https://www.youtube.com/watch?v=NbA_c5PoYMo&t=...
2,من التفكير سأكون سعيدا عندما أكون في مكان مختل...,121,133,وبدلا من التفكير سأكون سعيدا عندما أكون في مكا...,nvEvYgP8gWM,https://www.youtube.com/watch?v=nvEvYgP8gWM&t=...
3,فيها ثلاثة أو أربعة أشياء التي يجب أن أقوم بها...,330,336,لا أتجاوز فيها ثلاثة أو أربعة أشياء التي يجب أ...,FJqVUd8OQEU,https://www.youtube.com/watch?v=FJqVUd8OQEU&t=...
4,شيء وهو مهم جدا وهو أن لا تحاول تغيير الشخص ال...,134,147,ثاني شيء وهو مهم جدا وهو أن لا تحاول تغيير الش...,7ub86ZM9Z3k,https://www.youtube.com/watch?v=7ub86ZM9Z3k&t=...
...,...,...,...,...,...,...
695,هل أنت خائف ربما تعتقد بأنك لست جيد بما يكفي,3488,3493,هل أنت خائف ربما تعتقد بأنك لست جيد بما يكفي ل...,pvEcFKr3MQs,https://www.youtube.com/watch?v=pvEcFKr3MQs&t=...
696,إذا ذهبت إلى الصين أول شيء لديك إلى الأرض هو,660,664,إذا ذهبت إلى الصين أول شيء لديك إلى الأرض هو ا...,A4L4xsSE9TM,https://www.youtube.com/watch?v=A4L4xsSE9TM&t=...
697,هنا نكون قد وصلنا إلى نهاية الجزء الثاني قصة,365,370,وإلى هنا نكون قد وصلنا إلى نهاية الجزء الثاني ...,GgFsFNqeaqQ,https://www.youtube.com/watch?v=GgFsFNqeaqQ&t=...
698,مع أشخاص أعلم أنه يمكنني العمل معهم بشكل جيد,57,61,مع أشخاص أعلم أنه يمكنني العمل معهم بشكل جيد,LbGnGV5zxiQ,https://www.youtube.com/watch?v=LbGnGV5zxiQ&t=57s


In [11]:
df_10000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.csv")
df_10000 = df_10000.head(700)
df_10000

,search_string,start_time,end_time,sentence,video_id,video_url
0,لأن العلاقة بينك وبين الله بمجرد أن تغلق عليك ...,245,253,لأن العلاقة بينك وبين الله بمجرد أن تغلق عليك ...,pkIhkqWKMm8,https://www.youtube.com/watch?v=pkIhkqWKMm8&t=...
1,لن اقول لكم نادي الخامسة صباحا أو الثالثة او ا...,221,229,لن اقول لكم نادي الخامسة صباحا أو الثالثة او ا...,FJqVUd8OQEU,https://www.youtube.com/watch?v=FJqVUd8OQEU&t=...
2,لا تعيد منذ البداية و ربما تضيع فيه وقتا قد تق...,676,683,فكي لا تعيد منذ البداية و ربما تضيع فيه وقتا ق...,mnMDmVyr730,https://www.youtube.com/watch?v=mnMDmVyr730&t=...
3,ليس لديك أي رغبة على الإطلاق في مقابلة صديق صد...,148,161,ليس لديك أي رغبة على الإطلاق في مقابلة صديق صد...,WcKiIL31tqY,https://www.youtube.com/watch?v=WcKiIL31tqY&t=...
4,كنا نجد صعوبة كبيرة بالفعل في منافسة المناطق ا...,2920,2930,كنا نجد صعوبة كبيرة بالفعل في منافسة المناطق ا...,DlqJjdrYSi0,https://www.youtube.com/watch?v=DlqJjdrYSi0&t=...
...,...,...,...,...,...,...
695,الحدود إذا سيستغرق الأمر أربعة إلى خمسة أيام ل...,329,332,الحدود إذا سيستغرق الأمر أربعة إلى خمسة أيام ل...,xWGQoxMnwME,https://www.youtube.com/watch?v=xWGQoxMnwME&t=...
696,الامر ثم لذلك الخطوة الرابعة وهي خطوة تحديد او...,85,90,الامر ثم لذلك الخطوة الرابعة وهي خطوة تحديد او...,TJGEiE706Ds,https://www.youtube.com/watch?v=TJGEiE706Ds&t=85s
697,يعلم انك قبل ان توجد الى الدنيا لم تكن تعلم مت...,750,761,يعلم انك قبل ان توجد الى الدنيا لم تكن تعلم مت...,fFa0yGyd1kY,https://www.youtube.com/watch?v=fFa0yGyd1kY&t=...
698,إن العيادة ليس فيها سوى طبيبة واحدة لن تستطيع ...,2295,2298,إن العيادة ليس فيها سوى طبيبة واحدة لن تستطيع ...,9k6qMERUf4I,https://www.youtube.com/watch?v=9k6qMERUf4I&t=...


In [12]:
df_20000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.csv")
df_20000 = df_20000.head(700)
df_20000

,search_string,start_time,end_time,sentence,video_id,video_url
0,هذا شيء يجب ان أقوله لكم و أقوله لنفسي و لجميع...,238,253,هذا شيء يجب ان أقوله لكم و أقوله لنفسي و لجميع...,iFKU6f1vXnI,https://www.youtube.com/watch?v=iFKU6f1vXnI&t=...
1,وإن لم يكن ذلك ممكنا أو كنا نتحدث طوال الطريق ...,403,417,وإن لم يكن ذلك ممكنا أو كنا نتحدث طوال الطريق ...,QinTLIO8P6A,https://www.youtube.com/watch?v=QinTLIO8P6A&t=...
2,مثلا ربما يكون هدفك أن تصبح طبيبا لأنك تحمل قي...,333,352,مثلا ربما يكون هدفك أن تصبح طبيبا لأنك تحمل قي...,I789EC6uRI4,https://www.youtube.com/watch?v=I789EC6uRI4&t=...
3,لهذا أظن انه من الأشياء التي تجعلني أقف على أر...,170,187,لهذا أظن انه من الأشياء التي تجعلني أقف على أر...,U9s_ompNZ9c,https://www.youtube.com/watch?v=U9s_ompNZ9c&t=...
4,أول شيء احاول دائما أن أذكر نفسي به هو كم أنا ...,69,86,أول شيء احاول دائما أن أذكر نفسي به هو كم أنا ...,mnMDmVyr730,https://www.youtube.com/watch?v=mnMDmVyr730&t=69s
...,...,...,...,...,...,...
695,ان هو يعرض الشاشة لو انا ضغطت عليها كده يقول ل...,615,618,ان هو يعرض الشاشة لو انا ضغطت عليها كده يقول ل...,BBTIMRqq8ck,https://www.youtube.com/watch?v=BBTIMRqq8ck&t=...
696,هذه النقاط التي تكلمنا عنها النقطة رقم اثنان ا...,910,915,هذه النقاط التي تكلمنا عنها النقطة رقم اثنان ا...,EWcGATBR0d8,https://www.youtube.com/watch?v=EWcGATBR0d8&t=...
697,وهى لعبة بقاء ولكن بشكل مختلف قليلا عن العاب ا...,98,104,وهى لعبة بقاء ولكن بشكل مختلف قليلا عن العاب ا...,3HYSEeUEJoE,https://www.youtube.com/watch?v=3HYSEeUEJoE&t=98s
698,الجو الملكي البريطاني وذلك خلال دورية فوق البح...,229,236,الجو الملكي البريطاني وذلك خلال دورية فوق البح...,bHwkVSyMapI,https://www.youtube.com/watch?v=bHwkVSyMapI&t=...


In [13]:
df_40000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.csv")
df_40000 = df_40000.head(700)
df_40000

,search_string,start_time,end_time,sentence,video_id,video_url
0,هناك دراسة تشير إلى أنه لا يمكنك التواصل إلا م...,227,242,هناك دراسة تشير إلى أنه لا يمكنك التواصل إلا م...,_bfxALYv4A4,https://www.youtube.com/watch?v=_bfxALYv4A4&t=...
1,هل شعرت يوما بأنك عالق هل شعرت يوما و كأنك تضر...,9,24,هل شعرت يوما بأنك عالق هل شعرت يوما و كأنك تضر...,tEuQpz6cC0U,https://www.youtube.com/watch?v=tEuQpz6cC0U&t=9s
2,لذلك لا تركز على ما ليس لديك أو ما لا يمكنك فع...,639,658,لذلك لا تركز على ما ليس لديك أو ما لا يمكنك فع...,hVg78Xb6KlI,https://www.youtube.com/watch?v=hVg78Xb6KlI&t=...
3,صحيح لكن رياح القدر اليوم أطلقت دوامة غريبة من...,2572,2589,صحيح لكن رياح القدر اليوم أطلقت دوامة غريبة من...,DiM0xqXXFcE,https://www.youtube.com/watch?v=DiM0xqXXFcE&t=...
4,من خلال اختيارك التسكع مع الفاشلين ذوي النظرات...,439,456,من خلال اختيارك التسكع مع الفاشلين ذوي النظرات...,PX6Df4I95wA,https://www.youtube.com/watch?v=PX6Df4I95wA&t=...
...,...,...,...,...,...,...
695,بعد ساعتين تقريبا نقوم بطلاء وجه دهان مائي غير...,108,113,بعد ساعتين تقريبا نقوم بطلاء وجه دهان مائي غير...,LhNCAGYW0Zs,https://www.youtube.com/watch?v=LhNCAGYW0Zs&t=...
696,دائرة العقاب أكثر فأكثر ولم تعد تشمل الأحياء ف...,407,415,اتسعت دائرة العقاب أكثر فأكثر ولم تعد تشمل الأ...,ZkodPJ3xh5s,https://www.youtube.com/watch?v=ZkodPJ3xh5s&t=...
697,بكل بساطة لن تستطيع استثمار المال الخاص بك اذا...,184,189,بكل بساطة لن تستطيع استثمار المال الخاص بك اذا...,uK8ZXhDgpAY,https://www.youtube.com/watch?v=uK8ZXhDgpAY&t=...
698,كنت جدا متفاجئ شعرت بشعور جيد بعد البكاء شعرت ...,1765,1772,كنت جدا متفاجئ شعرت بشعور جيد بعد البكاء شعرت ...,kvsh5p5gbd4,https://www.youtube.com/watch?v=kvsh5p5gbd4&t=...


#### End Time Reassign

In [14]:
df_28_reassign = df_28.copy() # important because when it doesn't copy, main df are also change.
df_200_reassign = df_200.copy()
df_1000_reassign = df_1000.copy()
df_5000_reassign = df_5000.copy()
df_10000_reassign = df_10000.copy()
df_20000_reassign = df_20000.copy()
df_40000_reassign = df_40000.copy()

In [15]:
df_28_reassign["end_time"] = df_28_reassign["start_time"] + 1
df_200_reassign["end_time"] = df_200_reassign["start_time"] + 1
df_1000_reassign["end_time"] = df_1000_reassign["start_time"] + 1
df_5000_reassign["end_time"] = df_5000_reassign["start_time"] + 1
df_10000_reassign["end_time"] = df_10000_reassign["start_time"] + 1
df_20000_reassign["end_time"] = df_20000_reassign["start_time"] + 1
df_40000_reassign["end_time"] = df_40000_reassign["start_time"] + 1

In [16]:
out_path = "/home/kurubal/Downloads"

In [17]:
df_28_reassign.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.xlsx", index=False)
df_200_reassign.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.xlsx", index=False)
df_1000_reassign.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.xlsx", index=False)
df_5000_reassign.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.xlsx", index=False)
df_10000_reassign.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.xlsx", index=False)
df_20000_reassign.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.xlsx", index=False)
df_40000_reassign.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.xlsx", index=False)

In [ ]:
# !!!!! End Time Reassign files result check on youtube tool and change file with youtube tool file result as same name (old file should remove) !!!!! 

#### End Time Reassign Convert Orjinal (From Youtube Tool Result)

In [17]:
df_28_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.xlsx")
df_28_video_list = list(set(df_28_youtube_tool["video_id"]))

In [18]:
df_200_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.xlsx")
df_200_video_list = list(set(df_200_youtube_tool["video_id"]))

In [19]:
df_1000_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.xlsx")
df_1000_video_list = list(set(df_1000_youtube_tool["video_id"]))

In [20]:
df_5000_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.xlsx")
df_5000_video_list = list(set(df_5000_youtube_tool["video_id"]))

In [21]:
df_10000_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.xlsx")
df_10000_video_list = list(set(df_10000_youtube_tool["video_id"]))

In [22]:
df_20000_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.xlsx")
df_20000_video_list = list(set(df_20000_youtube_tool["video_id"]))

In [23]:
df_40000_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.xlsx")
df_40000_video_list = list(set(df_40000_youtube_tool["video_id"]))

##### Video List In video_id 

In [24]:
df_28_new = df_28[df_28["video_id"].isin(df_28_video_list)]
#df_28_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.csv", index=False)
df_28_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.xlsx", index=False)
df_28_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,نعم ماذا عن هذا هنا نعم,381,391,نعم ماذا عن هذا هنا نعم,2l3UajCV1mI,https://www.youtube.com/watch?v=2l3UajCV1mI&t=...
1,هل كان هذا كل ما في,366,368,هل كان هذا كل ما في الأمر,RLydD3uP6NI,https://www.youtube.com/watch?v=RLydD3uP6NI&t=...
2,عن ماذا كان كل ذلك,262,265,عن ماذا كان كل ذلك,rVkXdGORB8U,https://www.youtube.com/watch?v=rVkXdGORB8U&t=...
3,في في هل هذه أنت,4218,4220,مرحبا في في هل هذه أنت,7ONEprXyxcs,https://www.youtube.com/watch?v=7ONEprXyxcs&t=...
4,أنا هنا يا في في,4868,4870,أنا هنا يا في في,7ONEprXyxcs,https://www.youtube.com/watch?v=7ONEprXyxcs&t=...
...,...,...,...,...,...,...
432,و لا,612,614,و لا,Mms3BuNkHBs,https://www.youtube.com/watch?v=Mms3BuNkHBs&t=...
433,و هو,53,58,و هو,KBaejvlMXfY,https://www.youtube.com/watch?v=KBaejvlMXfY&t=53s
434,و هو,172,175,و هو,VR6GurhR33A,https://www.youtube.com/watch?v=VR6GurhR33A&t=...
435,و في,53,56,و في,glrQ18CS24A,https://www.youtube.com/watch?v=glrQ18CS24A&t=53s


In [25]:
df_200_new = df_200[df_200["video_id"].isin(df_200_video_list)]
#df_200_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.csv", index=False)
df_200_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.xlsx", index=False)
df_200_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,نعم حسنا ليس هناك لا يوجد أحد هنا لا يمكنني,435,440,نعم حسنا ليس هناك لا يوجد أحد هنا لا يمكنني ال...,NCem4xje9Ss,https://www.youtube.com/watch?v=NCem4xje9Ss&t=...
1,ماذا ماذا ماذا أنت ما هي لا لا تفعل أنا آسف,882,887,ماذا ماذا ماذا أنت ما هي لا لا تفعل أنا آسف اا...,HnSK6tiiQZg,https://www.youtube.com/watch?v=HnSK6tiiQZg&t=...
2,أوه اللعنة أوه اللعنة ما الذي يحدث يا إلهي,1036,1039,أوه اللعنة أوه اللعنة ما الذي يحدث يا إلهي,VSx7oL5PC7I,https://www.youtube.com/watch?v=VSx7oL5PC7I&t=...
3,لقد قلت أنت لا يمكنك أن تفعل ذلك بي هناك,1242,1248,لقد قلت أنت لا يمكنك أن تفعل ذلك بي هناك أشياء,gqC0SbiQQc4,https://www.youtube.com/watch?v=gqC0SbiQQc4&t=...
4,لذلك أنا لا أعرف أنني أعتقد إذا كنت تريد,295,299,لذلك أنا لا أعرف أنني أعتقد إذا كنت تريد,gkhlMGmoTWE,https://www.youtube.com/watch?v=gkhlMGmoTWE&t=...
...,...,...,...,...,...,...
558,مع أنه في كل مكان,15,17,مع أنه في كل مكان,fXukytNqkt4,https://www.youtube.com/watch?v=fXukytNqkt4&t=15s
559,يا إلهي و سيدي و,731,739,يا إلهي و سيدي و ربي,IHEwci25a5Q,https://www.youtube.com/watch?v=IHEwci25a5Q&t=...
560,ما إذا كان كل ما,1294,1296,ما إذا كان كل ما تريده,AFqZbxxIGK4,https://www.youtube.com/watch?v=AFqZbxxIGK4&t=...
561,أنا هنا يا في في,4868,4870,أنا هنا يا في في,7ONEprXyxcs,https://www.youtube.com/watch?v=7ONEprXyxcs&t=...


In [26]:
df_1000_new = df_1000[df_1000["video_id"].isin(df_1000_video_list)]
#df_1000_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.csv", index=False)
df_1000_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.xlsx", index=False)
df_1000_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,أشياء يجب ألا تتحدث عنها مع الآخرين على الإطلا...,21,30,هذه سبعة أشياء يجب ألا تتحدث عنها مع الآخرين ع...,FF1qfIe0Mb0,https://www.youtube.com/watch?v=FF1qfIe0Mb0&t=21s
1,أن يعرف ما إذا كنت تحصل على ما يكفي من الطعام ...,2470,2475,أراد أخوك أن يعرف ما إذا كنت تحصل على ما يكفي ...,RLydD3uP6NI,https://www.youtube.com/watch?v=RLydD3uP6NI&t=...
2,حسنا ولكن لن تعمل فقط على نفسك اسمع يا رجل أنت...,1327,1331,حسنا ولكن لن تعمل فقط على نفسك اسمع يا رجل أنت...,Q4PgAY_qXKg,https://www.youtube.com/watch?v=Q4PgAY_qXKg&t=...
3,ولكن اليوم الأول أنت لا تحتاج كل هذا أنا أعرف ...,1873,1877,ولكن اليوم الأول أنت لا تحتاج كل هذا أنا أعرف ...,SJ0tF6gA0cM,https://www.youtube.com/watch?v=SJ0tF6gA0cM&t=...
4,عندما يكون لديك نفس طويل أو عندما تكون تعلم ان...,845,850,عندما يكون لديك نفس طويل أو عندما تكون تعلم ان...,lhfkYiCpTiU,https://www.youtube.com/watch?v=lhfkYiCpTiU&t=...
...,...,...,...,...,...,...
695,تأخذ القليل منا إلى هناك الآن,4094,4120,تأخذ القليل منا إلى هناك الآن,2l3UajCV1mI,https://www.youtube.com/watch?v=2l3UajCV1mI&t=...
696,هذه المرة الأمر صعب بعض الشيء,3641,3644,هذه المرة الأمر صعب بعض الشيء,7fyY6ByIQi0,https://www.youtube.com/watch?v=7fyY6ByIQi0&t=...
697,هذه الأشياء يمكنني القيام بها,2635,2637,هذه الأشياء يمكنني القيام بها,wA0GhxqsAyA,https://www.youtube.com/watch?v=wA0GhxqsAyA&t=...
698,عليهم الوصول إلى الجانب الآخر,2000,2003,عليهم الوصول إلى الجانب الآخر,kFyyHU1u2d4,https://www.youtube.com/watch?v=kFyyHU1u2d4&t=...


In [27]:
df_5000_new = df_5000[df_5000["video_id"].isin(df_5000_video_list)]
#df_5000_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.csv", index=False)
df_5000_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.xlsx", index=False)
df_5000_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,سواء كنت تعتقد أنك تستطيع ذلك أو إذا كنت تعتقد...,39,54,قال هنري فورد سواء كنت تعتقد أنك تستطيع ذلك أو...,tEuQpz6cC0U,https://www.youtube.com/watch?v=tEuQpz6cC0U&t=39s
1,التقدم في العمر ليس بالأمر الجيد دائما أغلب ال...,277,288,التقدم في العمر ليس بالأمر الجيد دائما أغلب ال...,NbA_c5PoYMo,https://www.youtube.com/watch?v=NbA_c5PoYMo&t=...
2,من التفكير سأكون سعيدا عندما أكون في مكان مختل...,121,133,وبدلا من التفكير سأكون سعيدا عندما أكون في مكا...,nvEvYgP8gWM,https://www.youtube.com/watch?v=nvEvYgP8gWM&t=...
3,فيها ثلاثة أو أربعة أشياء التي يجب أن أقوم بها...,330,336,لا أتجاوز فيها ثلاثة أو أربعة أشياء التي يجب أ...,FJqVUd8OQEU,https://www.youtube.com/watch?v=FJqVUd8OQEU&t=...
4,شيء وهو مهم جدا وهو أن لا تحاول تغيير الشخص ال...,134,147,ثاني شيء وهو مهم جدا وهو أن لا تحاول تغيير الش...,7ub86ZM9Z3k,https://www.youtube.com/watch?v=7ub86ZM9Z3k&t=...
...,...,...,...,...,...,...
695,هل أنت خائف ربما تعتقد بأنك لست جيد بما يكفي,3488,3493,هل أنت خائف ربما تعتقد بأنك لست جيد بما يكفي ل...,pvEcFKr3MQs,https://www.youtube.com/watch?v=pvEcFKr3MQs&t=...
696,إذا ذهبت إلى الصين أول شيء لديك إلى الأرض هو,660,664,إذا ذهبت إلى الصين أول شيء لديك إلى الأرض هو ا...,A4L4xsSE9TM,https://www.youtube.com/watch?v=A4L4xsSE9TM&t=...
697,هنا نكون قد وصلنا إلى نهاية الجزء الثاني قصة,365,370,وإلى هنا نكون قد وصلنا إلى نهاية الجزء الثاني ...,GgFsFNqeaqQ,https://www.youtube.com/watch?v=GgFsFNqeaqQ&t=...
698,مع أشخاص أعلم أنه يمكنني العمل معهم بشكل جيد,57,61,مع أشخاص أعلم أنه يمكنني العمل معهم بشكل جيد,LbGnGV5zxiQ,https://www.youtube.com/watch?v=LbGnGV5zxiQ&t=57s


In [28]:
df_10000_new = df_10000[df_10000["video_id"].isin(df_10000_video_list)]
#df_10000_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.csv", index=False)
df_10000_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.xlsx", index=False)
df_10000_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,لأن العلاقة بينك وبين الله بمجرد أن تغلق عليك ...,245,253,لأن العلاقة بينك وبين الله بمجرد أن تغلق عليك ...,pkIhkqWKMm8,https://www.youtube.com/watch?v=pkIhkqWKMm8&t=...
1,لن اقول لكم نادي الخامسة صباحا أو الثالثة او ا...,221,229,لن اقول لكم نادي الخامسة صباحا أو الثالثة او ا...,FJqVUd8OQEU,https://www.youtube.com/watch?v=FJqVUd8OQEU&t=...
2,لا تعيد منذ البداية و ربما تضيع فيه وقتا قد تق...,676,683,فكي لا تعيد منذ البداية و ربما تضيع فيه وقتا ق...,mnMDmVyr730,https://www.youtube.com/watch?v=mnMDmVyr730&t=...
3,ليس لديك أي رغبة على الإطلاق في مقابلة صديق صد...,148,161,ليس لديك أي رغبة على الإطلاق في مقابلة صديق صد...,WcKiIL31tqY,https://www.youtube.com/watch?v=WcKiIL31tqY&t=...
4,كنا نجد صعوبة كبيرة بالفعل في منافسة المناطق ا...,2920,2930,كنا نجد صعوبة كبيرة بالفعل في منافسة المناطق ا...,DlqJjdrYSi0,https://www.youtube.com/watch?v=DlqJjdrYSi0&t=...
...,...,...,...,...,...,...
695,الحدود إذا سيستغرق الأمر أربعة إلى خمسة أيام ل...,329,332,الحدود إذا سيستغرق الأمر أربعة إلى خمسة أيام ل...,xWGQoxMnwME,https://www.youtube.com/watch?v=xWGQoxMnwME&t=...
696,الامر ثم لذلك الخطوة الرابعة وهي خطوة تحديد او...,85,90,الامر ثم لذلك الخطوة الرابعة وهي خطوة تحديد او...,TJGEiE706Ds,https://www.youtube.com/watch?v=TJGEiE706Ds&t=85s
697,يعلم انك قبل ان توجد الى الدنيا لم تكن تعلم مت...,750,761,يعلم انك قبل ان توجد الى الدنيا لم تكن تعلم مت...,fFa0yGyd1kY,https://www.youtube.com/watch?v=fFa0yGyd1kY&t=...
698,إن العيادة ليس فيها سوى طبيبة واحدة لن تستطيع ...,2295,2298,إن العيادة ليس فيها سوى طبيبة واحدة لن تستطيع ...,9k6qMERUf4I,https://www.youtube.com/watch?v=9k6qMERUf4I&t=...


In [29]:
df_20000_new = df_20000[df_20000["video_id"].isin(df_20000_video_list)]
#df_20000_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.csv", index=False)
df_20000_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.xlsx", index=False)
df_20000_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,هذا شيء يجب ان أقوله لكم و أقوله لنفسي و لجميع...,238,253,هذا شيء يجب ان أقوله لكم و أقوله لنفسي و لجميع...,iFKU6f1vXnI,https://www.youtube.com/watch?v=iFKU6f1vXnI&t=...
1,وإن لم يكن ذلك ممكنا أو كنا نتحدث طوال الطريق ...,403,417,وإن لم يكن ذلك ممكنا أو كنا نتحدث طوال الطريق ...,QinTLIO8P6A,https://www.youtube.com/watch?v=QinTLIO8P6A&t=...
2,مثلا ربما يكون هدفك أن تصبح طبيبا لأنك تحمل قي...,333,352,مثلا ربما يكون هدفك أن تصبح طبيبا لأنك تحمل قي...,I789EC6uRI4,https://www.youtube.com/watch?v=I789EC6uRI4&t=...
3,لهذا أظن انه من الأشياء التي تجعلني أقف على أر...,170,187,لهذا أظن انه من الأشياء التي تجعلني أقف على أر...,U9s_ompNZ9c,https://www.youtube.com/watch?v=U9s_ompNZ9c&t=...
4,أول شيء احاول دائما أن أذكر نفسي به هو كم أنا ...,69,86,أول شيء احاول دائما أن أذكر نفسي به هو كم أنا ...,mnMDmVyr730,https://www.youtube.com/watch?v=mnMDmVyr730&t=69s
...,...,...,...,...,...,...
695,ان هو يعرض الشاشة لو انا ضغطت عليها كده يقول ل...,615,618,ان هو يعرض الشاشة لو انا ضغطت عليها كده يقول ل...,BBTIMRqq8ck,https://www.youtube.com/watch?v=BBTIMRqq8ck&t=...
696,هذه النقاط التي تكلمنا عنها النقطة رقم اثنان ا...,910,915,هذه النقاط التي تكلمنا عنها النقطة رقم اثنان ا...,EWcGATBR0d8,https://www.youtube.com/watch?v=EWcGATBR0d8&t=...
697,وهى لعبة بقاء ولكن بشكل مختلف قليلا عن العاب ا...,98,104,وهى لعبة بقاء ولكن بشكل مختلف قليلا عن العاب ا...,3HYSEeUEJoE,https://www.youtube.com/watch?v=3HYSEeUEJoE&t=98s
698,الجو الملكي البريطاني وذلك خلال دورية فوق البح...,229,236,الجو الملكي البريطاني وذلك خلال دورية فوق البح...,bHwkVSyMapI,https://www.youtube.com/watch?v=bHwkVSyMapI&t=...


In [30]:
df_40000_new = df_40000[df_40000["video_id"].isin(df_40000_video_list)]
#df_40000_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.csv", index=False)
df_40000_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.xlsx", index=False)
df_40000_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,هناك دراسة تشير إلى أنه لا يمكنك التواصل إلا م...,227,242,هناك دراسة تشير إلى أنه لا يمكنك التواصل إلا م...,_bfxALYv4A4,https://www.youtube.com/watch?v=_bfxALYv4A4&t=...
1,هل شعرت يوما بأنك عالق هل شعرت يوما و كأنك تضر...,9,24,هل شعرت يوما بأنك عالق هل شعرت يوما و كأنك تضر...,tEuQpz6cC0U,https://www.youtube.com/watch?v=tEuQpz6cC0U&t=9s
2,لذلك لا تركز على ما ليس لديك أو ما لا يمكنك فع...,639,658,لذلك لا تركز على ما ليس لديك أو ما لا يمكنك فع...,hVg78Xb6KlI,https://www.youtube.com/watch?v=hVg78Xb6KlI&t=...
3,صحيح لكن رياح القدر اليوم أطلقت دوامة غريبة من...,2572,2589,صحيح لكن رياح القدر اليوم أطلقت دوامة غريبة من...,DiM0xqXXFcE,https://www.youtube.com/watch?v=DiM0xqXXFcE&t=...
4,من خلال اختيارك التسكع مع الفاشلين ذوي النظرات...,439,456,من خلال اختيارك التسكع مع الفاشلين ذوي النظرات...,PX6Df4I95wA,https://www.youtube.com/watch?v=PX6Df4I95wA&t=...
...,...,...,...,...,...,...
695,بعد ساعتين تقريبا نقوم بطلاء وجه دهان مائي غير...,108,113,بعد ساعتين تقريبا نقوم بطلاء وجه دهان مائي غير...,LhNCAGYW0Zs,https://www.youtube.com/watch?v=LhNCAGYW0Zs&t=...
696,دائرة العقاب أكثر فأكثر ولم تعد تشمل الأحياء ف...,407,415,اتسعت دائرة العقاب أكثر فأكثر ولم تعد تشمل الأ...,ZkodPJ3xh5s,https://www.youtube.com/watch?v=ZkodPJ3xh5s&t=...
697,بكل بساطة لن تستطيع استثمار المال الخاص بك اذا...,184,189,بكل بساطة لن تستطيع استثمار المال الخاص بك اذا...,uK8ZXhDgpAY,https://www.youtube.com/watch?v=uK8ZXhDgpAY&t=...
698,كنت جدا متفاجئ شعرت بشعور جيد بعد البكاء شعرت ...,1765,1772,كنت جدا متفاجئ شعرت بشعور جيد بعد البكاء شعرت ...,kvsh5p5gbd4,https://www.youtube.com/watch?v=kvsh5p5gbd4&t=...


#### Copy Move And Delete

In [102]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_*_Word{file_ext}.*")
output_file2

['Italian_Word_Group_Max_In_Youtube_Sentence_28_Word6.xlsx',
 'Italian_Word_Group_Max_In_Youtube_Sentence_200_Word6.xlsx',
 'Italian_Word_Group_Max_In_Youtube_Sentence_1000_Word6.xlsx',
 'Italian_Word_Group_Max_In_Youtube_Sentence_5000_Word6.xlsx',
 'Italian_Word_Group_Max_In_Youtube_Sentence_10000_Word6.xlsx',
 'Italian_Word_Group_Max_In_Youtube_Sentence_20000_Word6.xlsx',
 'Italian_Word_Group_Max_In_Youtube_Sentence_40000_Word6.xlsx']

In [103]:
for l in output_file2:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [104]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass